# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


# Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [190]:
Step1 = data.groupby([data.ProductName,data.CustomerID]).agg({'Quantity':sum}).fillna(0)

In [191]:
Step1.head(3)

Quantity
ProductName               CustomerID          
Anchovy Paste - 56 G Tube 639                1
                          883                1
                          891                1

In [20]:
# Misma forma de generar la tabla
# df = data.groupby([data.ProductName,data.CustomerID]).agg({'Quantity':sum}).unstack().fillna(0)
# df2 = pd.crosstab(data.ProductName,data.CustomerID, values = data.Quantity, aggfunc='sum' ).fillna(0)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [364]:
Step2 = data.pivot_table(index= 'ProductName', columns='CustomerID', aggfunc={'Quantity':sum}).fillna(0)

In [367]:
Step2.head()

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube           0.0   1.0   0.0   0.0  ...   0.0  25.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0  ...  25.0  25.0   
Appetizer - Mushroom Tart           0.0   1.0   0.0   0.0  ...  25.0   0.0   
Appetizer - Sausage Rolls           0.0   0.0   0.0   0.0  ...   0.0   0.0   
Apricots - Dried                    0.0   0.0   0.0   0.0  ...   0.0  25.0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube           0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart           0.0   0.0   0.0   0.0   0.0   0.0  25.0   
Appetizer - Sausage Rolls           0.0   0.0  25.0  25.0  25.0   0.0  25.0   
Apricots - Dried                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube           0.0  
Appetizer - Mini Egg Roll, Shrimp   0.0  
Appetizer - Mushroom Tart           0.0  
Appetizer - Sausage Rolls           0.0  
Apricots - Dried                    0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [372]:
a = squareform(pdist(Step2.T,'euclidean'))
distances = pd.DataFrame(1/(1 + a), index=Step2.columns, columns = Step2.columns)
distances.index = distances.index.droplevel(0)
distances.columns = distances.columns.droplevel(0)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [165]:
distances.index.names

FrozenList([None, 'CustomerID'])

In [166]:
distances.index.levels[0]

Index(['Quantity'], dtype='object')

In [167]:
distances.index = distances.index.droplevel(0)

In [168]:
distances

Quantity                                                    \
CustomerID     33        200       264       356       412       464     
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

                                                    ...                      \
CustomerID     477       639       649       669    ...     97697     97753   
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   
...              ...       ...       ...       ...  ...       ...       ...   
97928       0.004534  0.004526  0.004528  0.004523  ...  0.003509  0.003509   
98069       0.004589  0.004581  0.004585  0.004592  ...  0.003509  0.003509   
98159       0.004366  0.004363  0.004369  0.004365  ...  0.003509  0.003565   
98185       0.004168  0.004176  0.004172  0.004161  ...  0.003311  0.003162   
98200       0.004338  0.004335  0.004329  0.004347  ...  0.003418  0.003345   

                                                                        \
CustomerID     97769     97793     97900     97928     98069     98159   
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   
...              ...       ...       ...       ...       ...       ...   
97928       0.003300  0.003653  0.003579  1.000000  0.003638  0.003418   
98069       0.003369  0.003509  0.003496  0.003638  1.000000  0.003469   
98159       0.003322  0.003431  0.003322  0.003418  0.003469  1.000000   
98185       0.003255  0.003406  0.003172  0.003255  0.003345  0.003277   
98200       0.003381  0.003444  0.003288  0.003245  0.003381  0.003266   

                                
CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  
...              ...       ...  
97928       0.003255  0.003245  
98069       0.003345  0.003381  
98159       0.003277  0.003266  
98185       1.000000  0.003288  
98200       0.003288  1.000000  

[1000 rows x 1000 columns]

In [169]:
distances.columns = distances.columns.droplevel(0)

In [170]:
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [185]:
similarities = distances[33].sort_values(ascending=False)[1:]

In [186]:
similarities

CustomerID
264      0.087047
3535     0.087047
3317     0.087047
2503     0.085983
3305     0.085638
           ...   
91777    0.004140
92492    0.004121
95017    0.004077
97324    0.004032
92255    0.004007
Name: 33, Length: 999, dtype: float64

In [182]:
Customer_33 = [i for i in similarities.head(5).index]

In [183]:
Customer_33

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [194]:
Step1

Quantity
ProductName               CustomerID          
Anchovy Paste - 56 G Tube 639                1
                          883                1
                          891                1
                          2566               1
                          2902               1
...                                        ...
Zucchini - Yellow         94910             25
                          95078             25
                          95372             50
                          96024             25
                          96666             25

[63628 rows x 1 columns]

In [221]:
# Step1[Step1.index.get_level_values(1) in Customer_33]
import numpy as np
Step5 = Step1[np.in1d(Step1.index.get_level_values(1), Customer_33)]

In [222]:
Step5.head()

,,Quantity
ProductName,CustomerID,
Anchovy Paste - 56 G Tube,3535,1
Appetizer - Mushroom Tart,2503,1
Appetizer - Sausage Rolls,2503,1
Apricots - Dried,3535,1
Apricots - Halves,264,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [278]:
Step6 = Step5.groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending = False)

In [279]:
Step6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [235]:
# Step7 = Step1[np.in1d(Step1.index.get_level_values(1), 33)]

In [325]:
Step7 = Step1[np.in1d(Step1.index.get_level_values(1), 33)].groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending=False)
# Step7.rename(columns={'Quatity': '33'}, inplace= True)

In [311]:
merge = pd.concat([Step6, Step7], axis=1, sort=False)
merge

,Quantity,Quantity
Butter - Unsalted,3.0,NaN
Wine - Ej Gallo Sierra Valley,3.0,NaN
Towels - Paper / Kraft,3.0,1.0
Soup - Campbells Bean Medley,3.0,NaN
Wine - Blue Nun Qualitatswein,3.0,NaN
...,...,...
Grapes - Red,NaN,1.0
Ice Cream Bar - Drumstick,NaN,1.0
Mangoes,NaN,1.0
Muffin Batt - Choc Chk,NaN,1.0


In [312]:
new = merge[merge.iloc[:,1].isnull()]

In [357]:
new.iloc[0:5,0].index

Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
       'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein',
       'Chicken - Soup Base'],
      dtype='object')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [327]:
recomendations = {}
CustomerIDs = data['CustomerID'].unique()

In [358]:
for customer in CustomerIDs:
    similarities = distances[customer].sort_values(ascending=False)[1:]
    mirror = [i for i in similarities.head(5).index]
    df = Step1[np.in1d(Step1.index.get_level_values(1), mirror)]
    df2 = df.groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending = False)
    df3 = Step1[np.in1d(Step1.index.get_level_values(1), customer)].groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending=False)
    merge = pd.concat([df2, df3], axis=1, sort=False)
    filtered = merge[merge.iloc[:,1].isnull()].iloc[0:5,0].index
    recomendations[customer] = filtered

In [359]:
recomendations

{61288: Index(['Jagermeister', 'Chicken - Soup Base', 'Lime Cordial - Roses',
        'Macaroons - Two Bite Choc', 'Flavouring - Orange'],
       dtype='object'),
 77352: Index(['Wine - Valpolicella Masi', 'Tahini Paste', 'Tuna - Salad Premix',
        'Chicken - Soup Base', 'Isomalt'],
       dtype='object'),
 40094: Index(['Bread - Italian Corn Meal Poly', 'Puree - Mocha',
        'Tuna - Salad Premix', 'Sherry - Dry', 'Beer - Sleemans Cream Ale'],
       dtype='object'),
 23548: Index(['Squid - Tubes / Tenticles 10/20', 'Sprouts - Baby Pea Tendrils',
        'Flavouring - Orange', 'Bread - Raisin Walnut Oval', 'Banana Turning'],
       dtype='object'),
 78981: Index(['Vanilla Beans', 'Lettuce - Frisee', 'Yogurt - Blueberry, 175 Gr',
        'Cinnamon Buns Sticky', 'Pop Shoppe Cream Soda'],
       dtype='object'),
 83106: Index(['Cocoa Butter', 'Cheese - Boursin, Garlic / Herbs', 'Garlic - Peeled',
        'Cheese - Mix', 'Sauce - Hollandaise'],
       dtype='object'),
 11253: Index(

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [362]:
Step9 = pd.DataFrame(recomendations).T

In [363]:
Step9

,0,1,2,3,4
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...
49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
41286,Veal - Inside,"Thyme - Lemon, Fresh",Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style
85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [375]:
distances = pd.DataFrame(1/(1 + squareform(pdist(Step2.T,'cosine'))), index=Step2.columns, columns = Step2.columns)
distances.index = distances.index.droplevel(0)
distances.columns = distances.columns.droplevel(0)
for customer in CustomerIDs:
    similarities = distances[customer].sort_values(ascending=False)[1:]
    mirror = [i for i in similarities.head(5).index]
    df = Step1[np.in1d(Step1.index.get_level_values(1), mirror)]
    df2 = df.groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending = False)
    df3 = Step1[np.in1d(Step1.index.get_level_values(1), customer)].groupby('ProductName').agg({'Quantity':sum}).sort_values(by='Quantity', ascending=False)
    merge = pd.concat([df2, df3], axis=1, sort=False)
    filtered = merge[merge.iloc[:,1].isnull()].iloc[0:5,0].index
    recomendations[customer] = filtered

In [377]:
Step10 = pd.DataFrame(recomendations).T

In [378]:
Step10

,0,1,2,3,4
61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
23548,Juice - Orange,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky
78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...
49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
41286,"Hickory Smoke, Liquid","Salsify, Organic",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves
85878,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Cheese - Parmesan Grated,Foam Dinner Plate,Cheese - Mix
68506,Flavouring - Orange,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing
